In [4]:
print("Welcome to the Concrete Mix Design as per IS 10262")

Type=input("Enter concrete type (Plain/RCC): ")
cond=input("Enter the exposure level (Mild, Moderate, Severe, Very Severe, Extreme): ")
condition=["mild","moderate","severe","very severe","extreme"]
pg=[15,20,20,25] #grade recommended as per exposure level Plain
rccg=[20,25,30,35,40] #grade recommended as per exposure level RCC

if Type.lower()=="plain":
    for i in range(1,5):
        if cond.lower()==condition[i]:
            print("Minimum grade of concrete should be M {0}".format(pg[i-1]))
if Type.lower()=="rcc":
    for i in range(0,5):
        if cond.lower()==condition[i]:
            print("Minimum grade of concrete should be M {0}".format(rccg[i]))

fck=float(input("Characteristic compressive strength at 28 day's in N/mm^2: ")) #Asking the mix strength
SD=input("Is standard deviation calculated? (Y/N): ") #asking the Standard Deviation for strength

g=[10,15,20,25,30,35,40,45,50,55]
sd=[3.5,3.5,4,4,5,5,5,5,5,5]

if SD.lower()=="y":
    SD=int(input("Enter the calculated standard deviation (N/mm2): "))
elif SD.lower()=="n": #Table 1
    for i in range(0,10):
        if fck==g[i]:
            SD=sd[i]
Fck=fck+1.65*SD #target strength
print("Target strength: {0} N/mm2".format(Fck))
                
print("Enter A, B, C or D for the type of corse aggregate:")
Tagg=input("A. Angular coarse aggregate "+ " B. Sub-angular coarse aggregate "+ " C. Gravel with some crushed particles " + " D. Rounded gravel ")
Agg=int(input("Enter the maximum nominal size of aggregate (mm): "))
Fagg=int(input("Enter the grading zone of fine aggregates as per IS 383 (1,2,3 or 4): "))
Slump=float(input("Enter the slump value (mm): "))
pump=input("Is the concrete pumpable (Y/N)? ")
SP=input("Is superplasticizer added? (Y/N): ")
if SP.lower()=="y":
    sp=input("Is the water content reduction due to superplasticizer calculated? (Y/N): ")
    if sp.lower()=="n":
        print("Assuming 20% reduction in water content due to superplasticizer.")
        sp=20 #20% default reduction in water content (kg) per cubic meter of concrete.
    elif sp.lower()=="y":
        sp=float(input("Enter the percent water content reduction due to superplasticizer: "))
else: sp=0

sgc=input("Is specific gravity of cement calculated? (Y/N): ")
if sgc.lower()=="y":
    sgc=float(input("Enter specific gravity of cement: "))
else:
    sgc=3.15
    print("Taking specific gravity of cement as 3.15") 

sgca=input("Is specific gravity of coarse aggregate calculated? (Y/N): ")
if sgca.lower()=="y":
    sgca=float(input("Enter specific gravity of coarse agregate: "))
else:
    sgca=2.74
    print("Taking specific gravity of coarse aggregate as 2.74")

sgfa=input("Is specific gravity of fine aggregate calculated? (Y/N): ")
if sgfa.lower()=="y":
    sgfa=float(input("Enter specific gravity of fine agregate: "))
else:
    sgfa=2.74
    print("Taking specific gravity of fine aggregate as 2.74")
    


print("")
#Estimation of water cement ratio
wcPC=[0.6,0.6,0.5,0.45,0.4] #Max water content for Plain concrete
wcRCC=[0.55,0.5,0.45,0.45,0.4] #Max water content for Reinforced concrete
minCPC=[220,240,250,260,280] #min cement content kg/m3 for PC
minCRCC=[300,300,320,340,360] #min cement content kg/m3 for RCC
mincementcontent=False #whether min cement condition is satisfied
maxcementcontent=False #whether max cement condition is satisfied
#now assume a water cement ratio from max water-cement ratio from T5 IS 456

for i in range(0,5):
        if Type.lower()=="plain" and cond==condition[i]:
            wc=0.8*wcPC[i] #assuming 80% of max water cement ratio
        elif Type.lower()=="rcc" and cond==condition[i]:
            wc=0.8*wcRCC[i] #assuming 80% of max water cement ratio

while mincementcontent == False & maxcementcontent == False: #J for loop for trial and error for water content    
    #print("WC = {0}".format(wc)) Remove comment to see trial water cement ratios
    ag=[10,20,40] #mm
    Maxwc=[208,186,165] #kg/m3
    TaggSub=[0,10,20,25]
    tagg=["a",'b',"c","d"]
    for j in range (0,4):
        if Tagg.lower()==tagg[j]:
            Aggcorrection=TaggSub[j] #aggcorrection is subtracted according to aggregate type
    for i in range(0,3):
        if ag[i]==Agg:
            Mxwc=Maxwc[i] - Aggcorrection #litre for 25-50mm slump

    if 25<=Slump<=50:
        watercontent=Mxwc
    elif Slump>50:
        watercontent=Mxwc+((Slump-50)/25)*(3/100)*Mxwc
    watercontent = watercontent*(1-sp/100) #subtraction of water due to superplasticizer
    #print("Estimated water content for %.2f mm slump per cubic meter of concrete is %.1f litres."%(Slump,watercontent))

    #calculation of cement content
    cc=watercontent/wc
    
    if Type.lower()=="rcc":
        for i in range(0,5):
            if condition[i].lower()==cond.lower():
                #print("The minimum cement content for {0} concrete in {2} exposure is {1} kg.".format(Type,minCRCC[i],cond))
                if cc>=minCRCC[i] and cc<=450:
                    mincementcontent=True
                    maxcementconent=True
                    
                elif cc<minCRCC[i]:
                    #print("Reduce water content.")
                    wc=0.95*wc #Reducing the water content when min cement is not met
                elif cc>450:
                    #print("Increase water content.")
                    wc=1.1*wc
                    
    if Type.lower()=="plain":
        for i in range(0,5):
            if condition[i].lower()==cond.lower():
                #print("The minimum cement content for {0} concrete in {2} exposure is {1} kg.".format(Type,minCPC[i],cond))
                if cc>=minCPC[i] and cc<=450:
                    mincementcontent=True
                    maxcementconent=True
                    
                elif cc<minCPC[i]:
                    #print("Reduce water content.")
                    wc=0.95*wc #Reducing the water content when min cement is not met
                elif cc>450:
                    #print("Increase water content.")
                    wc=1.1*wc
                    
#calculation of aggregates Table 3
#in T3 the entries for wc=0.5 are in following order: 10mm[Zone 1,2,3,4], 20mm[Zone 1,2,3,4], and 40mm[Zone 1,2,3,4].
CA=[10,20,40]

T3=[[0.44,0.46,0.48,0.50],[0.60,0.62,0.64,0.66],[0.69,0.71,0.73,0.75]]
for i in range(0,3):
    for j in range(0,4):
        if Agg==CA[i] and Fagg==j+1:            
            if wc==0.5:
                Vca=(T3[i][j]) #Proportion of volume of course aggregate

            else:
                Vca=((wc-0.5)/0.05)*(-0.01)+ T3[i][j]
                
if pump.lower()=="y":
    Vc=0.9*Vca #reduction of course aggregate for pumpable concrete
else: Vc=Vca
Vf=1-Vc
print("The volume of coarse aggregate is %.2f."%Vc)
print("The volume of fine aggregate is %.2f."%Vf)

#Mix calculations
#Taking volume of concrete as 1 cubic meter
VC=cc/(sgc*1000)#m3 #3.15 is the Sp.Gravity of cement; VC is the cement vol
VW=watercontent/1000 #m3
if SP.lower()=="y":
    MSP=cc*0.02
    VSP=cc*0.02/(1.145*1000)#m3 #Superplasticizer @7% cement content and SG=1.145
else:
    VSP=0
VA=1-(VC+VW+VSP) #volume of all aggregates
Mca=VA*Vc*sgca*1000 #kg #2.74 is the SG of course aggregates
Mfa=VA*Vf*sgfa*1000 #kg

for i in range(0,5):
            if condition[i].lower()==cond.lower():
                print("The minimum cement content for {0} concrete in {2} exposure is {1} kg/m3.".format(Type,minCRCC[i],cond))
print("The maximum cement content is 450 kg/m3.")
print("")
print("The mix proportion is as follows: ")
print("Cement: %.0f kg/m3"%cc)
if mincementcontent==True & maxcementcontent==True:
    print("The minimum and maximum cement content conditions are satisfied.")
print("Water: %.0f kg/m3"%watercontent)
print("Fine aggregate: %.0f kg/m3"%Mfa)
print("Course aggregate: %.0f kg/m3"%Mca)
if SP.lower()=="y":
    print("Superplasticizer: %.0f kg/m3"%MSP)
print("Water-cement ratio: %.2f"%wc)

#c:fa:ca
a=Vf/VC
b=Vc/VC
A=Mfa/cc
B=Mca/cc
print("The mix proportion (C:FA:CA) by volume is: 1:%.1f:%.1f"%(a,b))
print("The mix proportion (C:FA:CA) by mass is: 1:%.1f:%.1f"%(A,B))

Welcome to the Concrete Mix Design as per IS 10262
Enter concrete type (Plain/RCC): RCC
Enter the exposure level (Mild, Moderate, Severe, Very Severe, Extreme): SEVERE
Minimum grade of concrete should be M 30
Characteristic compressive strength at 28 day's in N/mm^2: 40
Is standard deviation calculated? (Y/N): N
Target strength: 48.25 N/mm2
Enter A, B, C or D for the type of corse aggregate:
A. Angular coarse aggregate  B. Sub-angular coarse aggregate  C. Gravel with some crushed particles  D. Rounded gravel A
Enter the maximum nominal size of aggregate (mm): 20
Enter the grading zone of fine aggregates as per IS 383 (1,2,3 or 4): 2
Enter the slump value (mm): 100
Is the concrete pumpable (Y/N)? N
Is superplasticizer added? (Y/N): N
Is specific gravity of cement calculated? (Y/N): N
Taking specific gravity of cement as 3.15
Is specific gravity of coarse aggregate calculated? (Y/N): N
Taking specific gravity of coarse aggregate as 2.74
Is specific gravity of fine aggregate calculated? (